# Создание агента с функциями на базе GigaChat

С помощью [нейросетевой модели GigaChat](https://developers.sber.ru/docs/ru/gigachat/overview) вы можете вызывать различные функции — инструменты, которые будет использовать модель для решения поставленной задачи.
Модель самостоятельно решает, когда и какой инструмент нужно использовать. 
Вызванные функции выполняются на стороне клиента.
Вы можете использовать функции как разработанные самостоятельно, так и доступные в GigaChain и сторонних библиотеках.

Для использования функций нужно создать агента с помощью модуля [`gigachat_functions_agent`](/libs/langchain/langchain/agents/gigachat_functions_agent/base.py).

Раздел содержит пример агента, который задает вопрос в поисковом сервисе DuckDuckGo и рисует ответ с помощью Ascii-графики. Итоговый агент не только умеет работать с разными инструментами, но и обладает памятью.

## Подготовка к работе

Перед началом работы установите необходимые зависимости:

In [ ]:
pip install --upgrade --quiet gigachain gigachain-community duckduckgo-search pyfiglet

## Агент с одной функцией

Для создания агента с функцией обращения к поисковому сервису DuckDuckGo:

1. Инициализируйте GigaChat.
2. Добавьте функцию поиска в массив инструментов доступных модели.
3. Передайте агенту модель и массив инструментов.

### Инициализация GigaChat

Для работы агента создайте экземпляр класса GigaChat.
В классе укажите модель, поддерживающую работу с функциями:

In [24]:
from langchain_community.chat_models.gigachat import GigaChat

giga = GigaChat(
    credentials="<авторизационные_данные>",
    scope="GIGACHAT_API_PERS",
    model="GigaChat",
    verify_ssl_certs=False,
)

Объект GigaChat принимает параметры:

- `credentials` — авторизационные данные для обмена сообщениями с GigaChat API. О том как получить атворизационные данные — в разделе [Быстрый старт](/ru/gigachat/individuals-quickstart).
- `scope` — необязательный параметр, в котором можно указать версию API, к которой нужно обратиться. Возможные значения:
  
  - `GIGACHAT_API_PERS` — версия API для физических лиц;
  - `GIGACHAT_API_CORP` — версия API для ИП и юрлиц.

  По умолчанию запросы передаются в версию для физических лиц.

- `model` — необязательный параметр, в котором можно явно задать [модель GigaChat](/ru/gigachat/models).
- `verify_ssl_certs` — необязательный параметр, с помощью которого можно отключить проверку [сертификатов НУЦ Минцифры](/ru/gigachat/certificates).

[Подробнее о параметрах GigaChat](https://github.com/ai-forever/gigachat).

### Добавление функции поиска

Добавьте адаптер поискового сервиса [DuckDuckGo](https://www.duckduckgo.com) в список инструментов, которые будет использовать модель.

Для этого импортируйте класс `DuckDuckGoSearchRun` и добавьте функцию `DuckDuckGoSearchRun()` в массив `tools`:



In [25]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()
tools = [search_tool]

### Создание и запуск агента

Инициализируйте агента. Передайте ему модель, которая будет вызывать функции, и массив доступных ей инструментов `tools`:

In [26]:
from langchain.agents import AgentExecutor, create_gigachat_functions_agent

agent = create_gigachat_functions_agent(giga, tools)

# AgentExecutor создает среду, в которой будет работать агент
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

Запустите агента с помощью функции `invoke()`:

In [27]:
agent_executor.invoke(
    {"input": "Найди текущий курс биткоина и напечатай только число"}
)["output"]



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'текущий курс биткоина'}`
responded: Ваш запрос принят. Идет обработка...

Используйте наш бесплатный конвертер для расчета BTC - USD. Текущий курс обмена BTC на USD составляет $67,214.77. Бесплатный конвертер в реальном времени на основе данных CoinMarketCap. Курс Биткоина, Bitcoin (BTC) — графики и цены. Создавайте оповещения и уведомления на цену биткоина онлайн в реальном времени. Курс Bitcoin к доллару США BTC/USD на графике онлайн. Курс Bitcoin сегодня: $63,224.57 за 1 BTC. Цена BTC в USD изменилась за последние 24 часа на +0.88% . Текущая рыночная капитализация Bitcoin — $1.25 трлн при суммарном ... Посмотрите технический анализ биткоина на сегодня. 🤑 Курс криптовалюты и другие актуальные данные. Прогноз цены btc на основе теханализа. Текущий курс биткоина (btc) к доллару на биржах ... Курс биткоина на 13.07.2024 г.: 57 738,08 $ 1 биткоинов в рублях - Изменение за последний месяц ...67214.77



'67214.77'

## Агент с несколькими функциями

Для создания агента, который кроме поиска будет возвращать текст в виде Ascii-графики, вам потребуется библиотека pyfiglet.

Библиотека умеет работать только с латинскими символами и цифрами.

Пример работы pyfiglet:

In [19]:
import pyfiglet

pyfiglet.print_figlet("Hello!", font="epic")

          _______  _        _        _______  _ 
|\     /|(  ____ \( \      ( \      (  ___  )( )
| )   ( || (    \/| (      | (      | (   ) || |
| (___) || (__    | |      | |      | |   | || |
|  ___  ||  __)   | |      | |      | |   | || |
| (   ) || (      | |      | |      | |   | |(_)
| )   ( || (____/\| (____/\| (____/\| (___) | _ 
|/     \|(_______/(_______/(_______/(_______)(_)
                                                



### Использование нескольких функций

Чтобы упростить создание функций из python-кода, используйте декоратор `@tool`.
Он преобразует любую функцию в инструмент, доступный модели для вызова.

:::note

Модель ориентируется как на название и описание функции, так и на описание и типы аргументов возвращаемого значения.
Чтобы модель правильно понимала, как нужно использовать инструмент, все значения функции нужно явно указать.

:::

Создайте функцию `draw_banner()`, которая будет возвращать результат поиска в виде Ascii-графики, и добавьте ее в новый массив инструментов `new_tools`:

In [20]:
from langchain.tools import tool


@tool
def draw_banner(number: str) -> str:
    """Рисует баннер с текстом результатов кода в виде Ascii-графики

    Args:
        number (str): Число, которое нужно нарисовать на баннере
    """
    pyfiglet.print_figlet(number, font="epic")
    return "Draw complete"


new_tools = [search_tool, draw_banner]

Инициализируйте и запустите агента с просьбой найти нужное значение и нарисовать его на баннере:

In [21]:
agent = create_gigachat_functions_agent(giga, new_tools)

agent_executor = AgentExecutor(
    agent=agent,
    tools=new_tools,
    verbose=True,
)

agent_executor.invoke(
    {
        "input": "Найди в интернете курс биткоина в долларах и нарисуй это число на банере."
    }
)["output"]



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'курс биткоина в долларах'}`


Используйте наш бесплатный конвертер для расчета BTC - USD. Текущий курс обмена BTC на USD составляет $67,214.77. Бесплатный конвертер в реальном времени на основе данных CoinMarketCap. Курс Биткоина, Bitcoin (BTC) — графики и цены. Создавайте оповещения и уведомления на цену биткоина онлайн в реальном времени. Узнайте сколько будет ₿1 Биткоин (btc) в Долларах США (usd) - на данный момент. Получите реальные курсы обмена, анализ и динамику изменений валютной пары на графике и в таблице. Онлайн конвертер Биткоин (btc) в Доллар (USD) ⚡ Перевести биткоин в доллар по сегодняшнему курсу Курс в банках Курс в обменниках Курс НБУ Стоимость 10000 биткоинов в долларах США на сегодня составляет 680 216 904,85 $ по данным ЦБ РФ, по сравнению со вчерашним днём курс валюты увеличился на 1,26% (на +855,76 $). Курс биткоина по отношению к доллару США на графике ...
Invoking: `draw_banne

'- вот банер с текущим курсом биткоина к доллару.'

## Добавление памяти

В созданном агенте отсутствуют данные о состоянии, таким образом, он не помнит предыдущие взаимодействия с пользователем.

Чтобы агент помнил, о чем пользователь разговаривал с ним, передайте историю сообщений в переменной `chat_history`:

In [22]:
from langchain_core.messages import AIMessage, HumanMessage

agent_executor = AgentExecutor(
    agent=agent,
    tools=new_tools,
    verbose=False,
)

agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(
                content="Привет! Запомни трех животных - слон, жираф, крокодил"
            ),
            AIMessage(content="Привет! Хорошо, я запомнил."),
        ],
        "input": "Что я просил тебя запомнить?",
    }
)

{'chat_history': [HumanMessage(content='Привет! Запомни трех животных - слон, жираф, крокодил'),
  AIMessage(content='Привет! Хорошо, я запомнил.')],
 'input': 'Что я просил тебя запомнить?',
 'output': 'Вы просили меня запомнить слона, жирафа и крокодила.'}

### Использование памяти в чате с агентом

Вы можете использовать память для сохранения истории и промежуточных результатов общения пользователя с агентом:

In [33]:
chat_history = []
while True:
    user_input = input("Вы: ")
    print(f"Пользователь: {user_input}")
    if user_input == "":
        break
    result = agent_executor.invoke(
        {
            "chat_history": chat_history,
            "input": user_input,
        }
    )
    chat_history.append(HumanMessage(content=user_input))
    chat_history.append(AIMessage(content=result["output"]))
    print(f"Агент: {result['output']}")

Пользователь: Меня зовут Вася


> Entering new AgentExecutor chain...
Здравствуйте, Вася! Я генеративная языковая модель, созданная разработчиками Сбера. Могу ли я чем-то вам помочь?

> Finished chain.
Агент: Здравствуйте, Вася! Я генеративная языковая модель, созданная разработчиками Сбера. Могу ли я чем-то вам помочь?
Пользователь: Как меня зовут?


> Entering new AgentExecutor chain...
Вас зовут Вася.

> Finished chain.
Агент: Вас зовут Вася.
Пользователь: 
